# Getting Started with Space Language

Добро пожаловать в Space Language — систему многоуровневого анализа дискретных и непрерывных структур!

## Что такое Space Language?

Space Language объединяет **10 компонентов** в единый пайплайн:

- 🔄 **L0**: Система переписывания (дискретная динамика)
- 🤖 **L1**: Машина Тьюринга (вычисления)
- 📊 **L2**: Графы и SCC (структура)
- 🔍 **L3**: AC-автоматы (перекрытия)
- �� **L4**: Макро-система (семантика)
- 📝 **L5**: Генерация текста (AST)
- ✅ **L6**: Верификация (Z3, Isabelle)
- 📐 **L7**: Математические подъёмы (геометрия, меры, категории)

В этом туториале мы начнём с основ — системы переписывания строк.

## 1. Настройка окружения

Убедитесь, что путь к модулям Space Language добавлен в sys.path:

In [1]:
import sys
from pathlib import Path

# Добавляем src в путь
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"✓ Project root: {project_root}")
print(f"✓ Source path: {src_path}")

✓ Project root: /home/catman/Yandex.Disk/cuckoo/z/reals/libs/Experiments/Space/Lang
✓ Source path: /home/catman/Yandex.Disk/cuckoo/z/reals/libs/Experiments/Space/Lang/src


## 2. Основы: Строки и Правила

Система переписывания состоит из:
- **Строк** (`String`) — последовательности символов
- **Правил** (`Rule`) — преобразования вида `левая часть → правая часть`

Создадим простую систему для унарной арифметики:

In [2]:
from rewriting import String, Rule

# Создание строк
s1 = String.from_str("00")  # Два нуля
s2 = String.from_str("|")   # Штрих
s3 = String.from_str("0")   # Один ноль

print("Строки:")
print(f"  s1 = {s1}")
print(f"  s2 = {s2}")
print(f"  s3 = {s3}")
print()

# Создание правил
rule1 = Rule(String.from_str("00"), String.from_str("|"))   # 00 → |
rule2 = Rule(String.from_str("||"), String.from_str("0"))   # || → 0

print("Правила:")
print(f"  R1: {rule1.left} → {rule1.right}")
print(f"  R2: {rule2.left} → {rule2.right}")

Строки:
  s1 = 00
  s2 = |
  s3 = 0

Правила:
  R1: 00 → |
  R2: || → 0


## 3. Движок переписывания

`RewritingEngine` применяет правила и исследует пространство достижимых строк:

In [3]:
from rewriting import RewritingEngine

# Создаём движок с правилами
rules = [rule1, rule2]
engine = RewritingEngine(rules)

print(f"✓ Движок создан с {len(rules)} правилами")

✓ Движок создан с 2 правилами


## 4. Анализ достижимости

Исследуем, какие строки достижимы из начальной строки `"0000"`:

In [4]:
# Достижимость
initial = String.from_str("0000")
print(f"Начальная строка: {initial}")
print()

# Достижимые строки (с ограничением на глубину)
reachable_levels = engine.bounded_reach(initial, depth=10)

# Собираем все строки
all_reachable = set()
for level_strings in reachable_levels.values():
    all_reachable.update(level_strings)

print(f"Найдено {len(all_reachable)} достижимых строк:")
for i, s in enumerate(sorted(all_reachable, key=str), 1):
    if i <= 10:  # Показываем первые 10
        print(f"  {i}. {s}")

Начальная строка: 0000

Найдено 6 достижимых строк:
  1. 0
  2. 0000
  3. 00|
  4. 0|0
  5. |00
  6. ||


## 5. Применение правил

Посмотрим, какие правила можно применить к строке:

In [5]:
# Нормальные формы (строки без применений)
normal_forms = {s for s in all_reachable if not engine.all_applications(s)}

print(f"Нормальные формы ({len(normal_forms)} строк):")
for nf in sorted(normal_forms, key=str):
    print(f"  {nf}")

Нормальные формы (2 строк):
  0
  0|0


## 6. Нормальные формы

Нормальная форма — строка, к которой нельзя применить ни одно правило:

In [6]:
# Последовательное применение правил
s = String.from_str("0000")
print(f"Начало: {s}")

# Применяем правила пока возможно
for step in range(5):
    apps = engine.all_applications(s)
    if not apps:
        print(f"Шаг {step}: Нормальная форма достигнута")
        break
    
    s, rule, pos = apps[0]  # Берём первое применение
    print(f"Шаг {step + 1}: {s} (правило: {rule} на позиции {pos})")

Начало: 0000
Шаг 1: |00 (правило: 00 → | на позиции 0)
Шаг 2: || (правило: 00 → | на позиции 1)
Шаг 3: 0 (правило: || → 0 на позиции 0)
Шаг 3: Нормальная форма достигнута


## 7. Omega-предел

ω-предел показывает долгосрочное поведение системы:

In [7]:
# Omega-предел (циклические аттракторы)
omega = engine.omega_limit(String.from_str("0000"), max_steps=20)

print(f"ω-предел ({len(omega)} строк):")
for s in sorted(omega, key=str):
    print(f"  {s}")
print()
print("Это предельное множество, достижимое при бесконечном применении правил")

ω-предел (1 строк):
  0

Это предельное множество, достижимое при бесконечном применении правил


## 📊 Резюме

В этом туториале вы узнали:

✅ Как создавать строки и правила  
✅ Как использовать `RewritingEngine`  
✅ Как находить достижимые строки с `bounded_reach`  
✅ Как определять нормальные формы  
✅ Как вычислять ω-предел системы  

## 🚀 Следующие шаги

Переходите к следующим туториалам:

- **02_graph_analysis.ipynb** — Построение графов G_L и SCC анализ
- **03_mathematical_lifts.ipynb** — Геометрические и мерные подъёмы
- **04_full_integration.ipynb** — Использование полного пайплайна